In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

from tqdm.auto import tqdm

from simulator import *

In [2]:
plt.rcParams['figure.figsize'] = [15, 10]

In [3]:
sample_size = 200000
catalog_size = 100
num_caches = 3
num_dest = 4
relative_cache_size = 0.3 / num_caches
cache_sizes = np.ones(num_caches, dtype=int)*int(relative_cache_size*catalog_size)

In [4]:
trace = gen_irm_trace(sample_size, catalog_size, power_law_exp=0.8)
destinations = gen_dest_trace(sample_size, num_dest)

In [5]:
# trace, catalog_size, sample_size = parse_trace(load_mat_array(r"./traces/yt_trace.mat"))

# num_caches = 3
# num_dest = 4
# relative_cache_size = 0.3 / num_caches
# cache_sizes = np.ones(num_caches, dtype=int)*int(relative_cache_size*catalog_size)
# destinations = np.random.randint(low=0, high=num_dest, size=sample_size)

In [6]:
# Generate Utilities and caches
utilities = np.array([
    [1, 2, 100],
    [1, 2, 100],
    [1, 2, 100],
    [1, 2, 100]
])
edges = np.array([
    [1, 1, 0],
    [1, 1, 0],
    [0, 1, 1],
    [0, 1, 1]
]) # destinations x caches

In [7]:
caches_init = np.array([init_cache(cache_size, catalog_size) for cache_size in cache_sizes])

In [8]:
# Initialize caches
cache_mLRU = mLRU(cache_sizes=cache_sizes, catalog_size=catalog_size, caches_init=caches_init, utilities=utilities, edges=edges)
cache_LazyLRU = LazyLRU(cache_sizes=cache_sizes, catalog_size=catalog_size, caches_init=caches_init, utilities=utilities, edges=edges)
cache_BH = femtoBH(cache_sizes=cache_sizes, catalog_size=catalog_size, caches_init=caches_init, utilities=utilities, edges=edges, trace=trace, destinations=destinations)
cache_BSA = BSA(cache_sizes, catalog_size, sample_size, utilities, edges, caches_init=caches_init)
cache_D_BSA = DBSA(cache_sizes, catalog_size, sample_size, utilities, edges, caches_init=caches_init)


RankOGA_init = lambda cache_size, catalog_size, cache_init : RankOGA(cache_size, catalog_size, cache_init, sample_size=sample_size)
RankFTPL_init = lambda cache_size, catalog_size, cache_init : RankFTPL(cache_size, catalog_size, cache_init, sample_size=sample_size)
ranking_expert_policies = (RankLRU, RankFTPL_init, RankOGA_init)

cache_DEC = FemtoDEC(cache_sizes, catalog_size, caches_init, utilities, edges, ranking_expert_policies, mixing=False)
cache_DEC_mix = FemtoDEC(cache_sizes, catalog_size, caches_init, utilities, edges, ranking_expert_policies, mixing=True)



In [9]:
caches = (cache_DEC, cache_DEC_mix, cache_BSA, cache_D_BSA, cache_mLRU, cache_LazyLRU, cache_BH)
caches = simulate_caches_femto_parallel(caches, trace, destinations)
# save_caches(caches, f"./figures/femto_irm_caches.pickle")
(cache_DEC, cache_DEC_mix, cache_BSA, cache_D_BSA, cache_mLRU, cache_LazyLRU, cache_BH) = caches

# for request, dest in tqdm(zip(trace, destinations), total=len(trace)):
#     cache_mLRU.request(request, dest)
#     cache_LazyLRU.request(request, dest)
#     cache_BH.request(request, dest)
#     cache_BSA.request(request, dest)
#     cache_D_BSA.request(request, dest)
#     cache_DEC.request(request, dest)
#     cache_DEC_mix.request(request, dest)

  0%|          | 0/7 [00:00<?, ?it/s]

In [10]:
%matplotlib ipympl
plt.plot(cache_mLRU.get_hitrate())
plt.plot(cache_LazyLRU.get_hitrate())
plt.plot(cache_BH.get_hitrate())
plt.plot(cache_BSA.get_hitrate())
plt.plot(cache_D_BSA.get_hitrate())
plt.plot(cache_DEC.get_hitrate())
plt.plot(cache_DEC_mix.get_hitrate())
plt.ylabel("Utility")
plt.xlabel("Time")
plt.legend(["mLRU", "LazyLRU", "~BH", "BSA", "D-BSA", "FemtoDEC", "FemtoDEC (Mix)"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/koen/anaconda3/lib/python3.7/site-packages/ipympl/backend_nbagg.py:180: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  self.manager.handle_json(content)


In [11]:
trace = random_replacement_model(sample_size, catalog_size, power_law_exp=0.8)
destinations = gen_dest_trace(sample_size, num_dest)

In [12]:
caches_init = np.array([init_cache(cache_size, catalog_size) for cache_size in cache_sizes])

In [13]:
# Initialize caches
cache_mLRU = mLRU(cache_sizes=cache_sizes, catalog_size=catalog_size, caches_init=caches_init, utilities=utilities, edges=edges)
cache_LazyLRU = LazyLRU(cache_sizes=cache_sizes, catalog_size=catalog_size, caches_init=caches_init, utilities=utilities, edges=edges)
cache_BH = femtoBH(cache_sizes=cache_sizes, catalog_size=catalog_size, caches_init=caches_init, utilities=utilities, edges=edges, trace=trace, destinations=destinations)
cache_BSA = BSA(cache_sizes, catalog_size, sample_size, utilities, edges, caches_init=caches_init)
cache_D_BSA = DBSA(cache_sizes, catalog_size, sample_size, utilities, edges, caches_init=caches_init)


RankOGA_init = lambda cache_size, catalog_size, cache_init : RankOGA(cache_size, catalog_size, cache_init, sample_size=sample_size)
RankFTPL_init = lambda cache_size, catalog_size, cache_init : RankFTPL(cache_size, catalog_size, cache_init, sample_size=sample_size)
ranking_expert_policies = (RankLRU, RankFTPL_init, RankOGA_init)

cache_DEC = FemtoDEC(cache_sizes, catalog_size, caches_init, utilities, edges, ranking_expert_policies, mixing=False)
cache_DEC_mix = FemtoDEC(cache_sizes, catalog_size, caches_init, utilities, edges, ranking_expert_policies, mixing=True)



In [14]:
caches = (cache_DEC, cache_DEC_mix, cache_BSA, cache_D_BSA, cache_mLRU, cache_LazyLRU, cache_BH)
caches = simulate_caches_femto_parallel(caches, trace, destinations)
save_caches(caches, f"./figures/femto_rrm_caches.pickle")
(cache_DEC, cache_DEC_mix, cache_BSA, cache_D_BSA, cache_mLRU, cache_LazyLRU, cache_BH) = caches

  0%|          | 0/7 [00:00<?, ?it/s]

In [15]:
%matplotlib ipympl
plt.plot(cache_mLRU.get_hitrate())
plt.plot(cache_LazyLRU.get_hitrate())
plt.plot(cache_BH.get_hitrate())
plt.plot(cache_BSA.get_hitrate())
plt.plot(cache_D_BSA.get_hitrate())
plt.plot(cache_DEC.get_hitrate())
plt.plot(cache_DEC_mix.get_hitrate())
plt.ylabel("Utility")
plt.xlabel("Time")
plt.legend(["mLRU", "LazyLRU", "~BH", "BSA", "D-BSA", "FemtoDEC", "FemtoDEC (Mix)"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/koen/anaconda3/lib/python3.7/site-packages/ipympl/backend_nbagg.py:180: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  self.manager.handle_json(content)


In [16]:
trace, catalog_size, sample_size = parse_trace(load_mat_array(r"./traces/yt_trace.mat"))

cache_sizes = np.ones(num_caches, dtype=int)*int(relative_cache_size*catalog_size)
destinations = gen_dest_trace(sample_size, num_dest)

In [20]:
# Initialize caches
caches_init = np.array([init_cache(cache_size, catalog_size) for cache_size in cache_sizes])

cache_mLRU = mLRU(cache_sizes=cache_sizes, catalog_size=catalog_size, caches_init=caches_init, utilities=utilities, edges=edges)
cache_LazyLRU = LazyLRU(cache_sizes=cache_sizes, catalog_size=catalog_size, caches_init=caches_init, utilities=utilities, edges=edges)
cache_BH = femtoBH(cache_sizes=cache_sizes, catalog_size=catalog_size, caches_init=caches_init, utilities=utilities, edges=edges, trace=trace, destinations=destinations)
cache_BSA = BSA(cache_sizes, catalog_size, sample_size, utilities, edges, caches_init=caches_init)
cache_D_BSA = DBSA(cache_sizes, catalog_size, sample_size, utilities, edges, caches_init=caches_init)


RankOGA_init = lambda cache_size, catalog_size, cache_init : RankOGA(cache_size, catalog_size, cache_init, sample_size=sample_size)
RankFTPL_init = lambda cache_size, catalog_size, cache_init : RankFTPL(cache_size, catalog_size, cache_init, sample_size=sample_size)
ranking_expert_policies = (RankLRU, RankFTPL_init, RankOGA_init)

cache_DEC = FemtoDEC(cache_sizes, catalog_size, caches_init, utilities, edges, ranking_expert_policies, mixing=False)
cache_DEC_mix = FemtoDEC(cache_sizes, catalog_size, caches_init, utilities, edges, ranking_expert_policies, mixing=True)



In [21]:
caches = (cache_DEC, cache_DEC_mix, cache_BSA, cache_D_BSA, cache_mLRU, cache_LazyLRU, cache_BH)
caches = simulate_caches_femto(caches, trace, destinations)
save_caches(caches, f"./figures/femto_yt_caches.pickle")
(cache_DEC, cache_DEC_mix, cache_BSA, cache_D_BSA, cache_mLRU, cache_LazyLRU, cache_BH) = caches

  0%|          | 0/100000 [00:00<?, ?it/s]

In [22]:
%matplotlib ipympl
plt.plot(cache_mLRU.get_hitrate())
plt.plot(cache_LazyLRU.get_hitrate())
plt.plot(cache_BH.get_hitrate())
plt.plot(cache_BSA.get_hitrate())
plt.plot(cache_D_BSA.get_hitrate())
plt.plot(cache_DEC.get_hitrate())
plt.plot(cache_DEC_mix.get_hitrate())
plt.ylabel("Utility")
plt.xlabel("Time")
plt.legend(["mLRU", "LazyLRU", "~BH", "BSA", "D-BSA", "FemtoDEC", "FemtoDEC (Mix)"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/koen/anaconda3/lib/python3.7/site-packages/ipympl/backend_nbagg.py:180: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  self.manager.handle_json(content)


In [7]:
trace = get_movielens_trace()
sample_size, catalog_size = get_trace_stats(trace)

cache_sizes = np.ones(num_caches, dtype=int)*int(relative_cache_size*catalog_size)
destinations = gen_dest_trace(sample_size, num_dest)

In [8]:
# Initialize caches
caches_init = np.array([init_cache(cache_size, catalog_size) for cache_size in cache_sizes])

cache_mLRU = mLRU(cache_sizes=cache_sizes, catalog_size=catalog_size, caches_init=caches_init, utilities=utilities, edges=edges)
cache_LazyLRU = LazyLRU(cache_sizes=cache_sizes, catalog_size=catalog_size, caches_init=caches_init, utilities=utilities, edges=edges)
cache_BH = femtoBH(cache_sizes=cache_sizes, catalog_size=catalog_size, caches_init=caches_init, utilities=utilities, edges=edges, trace=trace, destinations=destinations)
cache_BSA = BSA(cache_sizes, catalog_size, sample_size, utilities, edges, caches_init=caches_init)
cache_D_BSA = DBSA(cache_sizes, catalog_size, sample_size, utilities, edges, caches_init=caches_init)


RankOGA_init = lambda cache_size, catalog_size, cache_init : RankOGA(cache_size, catalog_size, cache_init, sample_size=sample_size)
RankFTPL_init = lambda cache_size, catalog_size, cache_init : RankFTPL(cache_size, catalog_size, cache_init, sample_size=sample_size)
ranking_expert_policies = (RankLRU, RankFTPL_init, RankOGA_init)

cache_DEC = FemtoDEC(cache_sizes, catalog_size, caches_init, utilities, edges, ranking_expert_policies, mixing=False)
cache_DEC_mix = FemtoDEC(cache_sizes, catalog_size, caches_init, utilities, edges, ranking_expert_policies, mixing=True)


In [9]:
caches = (cache_DEC, cache_DEC_mix, cache_BSA, cache_D_BSA, cache_mLRU, cache_LazyLRU, cache_BH)
caches = simulate_caches_femto(caches, trace, destinations)
save_caches(caches, f"./figures/femto_movielens_caches.pickle")
(cache_DEC, cache_DEC_mix, cache_BSA, cache_D_BSA, cache_mLRU, cache_LazyLRU, cache_BH) = caches

  0%|          | 0/1000209 [00:00<?, ?it/s]

In [10]:
%matplotlib ipympl
plt.plot(cache_mLRU.get_hitrate())
plt.plot(cache_LazyLRU.get_hitrate())
plt.plot(cache_BH.get_hitrate())
plt.plot(cache_BSA.get_hitrate())
plt.plot(cache_D_BSA.get_hitrate())
plt.plot(cache_DEC.get_hitrate())
plt.plot(cache_DEC_mix.get_hitrate())
plt.ylabel("Utility")
plt.xlabel("Time")
plt.legend(["mLRU", "LazyLRU", "~BH", "BSA", "D-BSA", "FemtoDEC", "FemtoDEC (Mix)"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/koen/anaconda3/lib/python3.7/site-packages/ipympl/backend_nbagg.py:180: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  self.manager.handle_json(content)
